# DSII — Retail Customer Segmentation & Anomaly Detection


In [1]:
# Paths and Directories
from pathlib import Path
BASE  = Path.cwd()
DATA  = BASE / "data"
PROC  = DATA / "processed"
FIGS  = BASE / "reports" / "figures"

for p in [DATA, PROC, FIGS]:
    p.mkdir(parents=True, exist_ok=True)

DATASET_CSV = BASE / "online_retail_II.csv"  


In [2]:
# Setup Enviornment
import os, random, json, math, time, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

RANDOM_SEED = 1337
np.random.seed(RANDOM_SEED); random.seed(RANDOM_SEED); os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)

# Paths (relative to notebook)
ROOT = Path(".").resolve()
DATASET_PATH = ROOT / "online_retail_II.csv"  # <-- keep CSV next to notebook
PROC = ROOT / "data" / "processed"
FIGS = ROOT / "reports" / "figures"
PROC.mkdir(parents=True, exist_ok=True); FIGS.mkdir(parents=True, exist_ok=True)

CFG = {
    "random_seed": RANDOM_SEED,
    "paths": {"processed_dir": str(PROC), "figures_dir": str(FIGS)},
    "clustering": {"kmeans": {"kmin": 2, "kmax": 10, "n_init": 10}},
    "projection": {"method": "umap", "random_state": RANDOM_SEED},
    "seq": {"seq_len": 50, "max_vocab": 2000, "min_count": 5, "batch": 256, "epochs": 10},
    "ae": {"bottleneck": 8, "l2": 1e-4, "dropout": 0.10, "epochs": 200, "batch": 256, "patience": 10},
}

print("Config OK →", json.dumps(CFG, indent=2))
print("Data:", DATASET_PATH)
print("Outputs →", PROC, "and", FIGS)

Config OK → {
  "random_seed": 1337,
  "paths": {
    "processed_dir": "C:\\Users\\mttng\\Downloads\\retail-segmentation-anomalies\\data\\processed",
    "figures_dir": "C:\\Users\\mttng\\Downloads\\retail-segmentation-anomalies\\reports\\figures"
  },
  "clustering": {
    "kmeans": {
      "kmin": 2,
      "kmax": 10,
      "n_init": 10
    }
  },
  "projection": {
    "method": "umap",
    "random_state": 1337
  },
  "seq": {
    "seq_len": 50,
    "max_vocab": 2000,
    "min_count": 5,
    "batch": 256,
    "epochs": 10
  },
  "ae": {
    "bottleneck": 8,
    "l2": 0.0001,
    "dropout": 0.1,
    "epochs": 200,
    "batch": 256,
    "patience": 10
  }
}
Data: C:\Users\mttng\Downloads\retail-segmentation-anomalies\online_retail_II.csv
Outputs → C:\Users\mttng\Downloads\retail-segmentation-anomalies\data\processed and C:\Users\mttng\Downloads\retail-segmentation-anomalies\reports\figures


In [3]:
# Utility pathing for figures & timers
def savefig(name, tight=True, dpi=200):
    if tight: plt.tight_layout()
    out = Path(CFG["paths"]["figures_dir"]) / name
    plt.savefig(out, dpi=dpi)
    plt.close()
    return out

from contextlib import contextmanager
@contextmanager
def timer(msg):
    import time
    t0 = time.time()
    print(f"[start] {msg}")
    yield
    print(f"[done]  {msg} in {time.time()-t0:.2f}s")

def ensure_cols(df):
    # tolerant column mapper
    cols = {c.lower().strip(): c for c in df.columns}
    def find(*cands):
        for k in cands:
            if k in cols: return cols[k]
        return None
    mapping = {
        "Invoice": find("invoice","invoiceno","invoice no","invoicenumber"),
        "InvoiceDate": find("invoicedate","date"),
        "CustomerID": find("customer id","customerid","custid","cust id"),
        "StockCode": find("stockcode","sku","productcode","product code"),
        "Quantity": find("quantity","qty","count"),
        "Price": find("price","unitprice","unit price"),
        "Description": find("description","desc")
    }
    missing = [k for k,v in mapping.items() if v is None and k not in ["Description"]]
    if missing:
        raise KeyError(f"Missing required columns: {missing}. Found: {list(df.columns)}")
    return mapping


In [4]:
# ---- Poster style (once per notebook) ----
import matplotlib as mpl
mpl.rcParams.update({
    "figure.dpi": 300,
    "savefig.dpi": 300,
    "axes.titlesize": 18,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "font.size": 12,
    "axes.grid": True,
    "grid.alpha": 0.25,
    "grid.linestyle": "--",
    "figure.facecolor": "white",
    "axes.facecolor": "white"
})


In [5]:
# Loading & Cleaning Data
with timer("Load CSV"):
    raw = pd.read_csv(DATASET_PATH, encoding="utf-8", low_memory=False)

colmap = ensure_cols(raw)
raw = raw.rename(columns={v:k for k,v in colmap.items() if v is not None})

# Parse dates, compute line totals
raw["InvoiceDate"] = pd.to_datetime(raw["InvoiceDate"], errors="coerce")
raw = raw.dropna(subset=["InvoiceDate"])
raw["LineTotal"] = raw["Quantity"] * raw["Price"]

# Remove cancellations/returns (Invoice starts with 'C'), invalid rows
mask_cancel = raw["Invoice"].astype(str).str.startswith("C")
raw = raw.loc[~mask_cancel].copy()
raw = raw[(raw["Quantity"] > 0) & (raw["Price"] > 0)]
raw = raw.dropna(subset=["CustomerID"])
raw["CustomerID"] = raw["CustomerID"].astype(str)

print("Rows after cleaning:", len(raw))
print("Unique customers:", raw["CustomerID"].nunique())
print("Date range:", raw["InvoiceDate"].min(), "→", raw["InvoiceDate"].max())


[start] Load CSV
[done]  Load CSV in 0.86s
Rows after cleaning: 805549
Unique customers: 5878
Date range: 2009-12-01 07:45:00 → 2011-12-09 12:50:00


In [6]:
# Feature Engineering (RFM + interpretable features)
import pandas as pd
import numpy as np
from pathlib import Path

# 1. Load CSV
CSV_PATH = Path(globals().get("DATASET_PATH", "online_retail_II.csv"))
raw = pd.read_csv(CSV_PATH, encoding_errors="ignore")

# 2. Harmonize column names across variants
def rename_first_match(df, std, candidates):
    for c in candidates:
        if c in df.columns:
            return df if c == std else df.rename(columns={c: std})
    raise KeyError(f"Required column '{std}' not found. Looked for {candidates}. Got {list(df.columns)[:20]} ...")

df = raw.copy()
df = rename_first_match(df, "CustomerID",  ["CustomerID","Customer ID","customerid"])
df = rename_first_match(df, "InvoiceNo",   ["InvoiceNo","Invoice","Invoice No"])
df = rename_first_match(df, "InvoiceDate", ["InvoiceDate","Invoice Date","date","Date"])
df = rename_first_match(df, "Price",       ["Price","UnitPrice","Unit Price"])
df = rename_first_match(df, "Quantity",    ["Quantity","Qty","quantity"])
df = rename_first_match(df, "StockCode",   ["StockCode","Stock Code","SKU","ProductCode","Product Code","Description"])

# 3. Strong typing & cleaning
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
df["InvoiceNo"]   = df["InvoiceNo"].astype(str).str.strip()
df["CustomerID"]  = pd.to_numeric(df["CustomerID"], errors="coerce").round().astype("Int64")

# Drop cancels/returns (InvoiceNo that start with 'C'), invalid rows, and nulls
before = len(df)
df = df[~df["InvoiceNo"].str.startswith("C", na=False)]
df = df.dropna(subset=["CustomerID","InvoiceDate"]).copy()
df = df[(df["Quantity"] > 0) & (df["Price"] > 0)].copy()
df["CustomerID"] = df["CustomerID"].astype("int64")

# 4. Line total
if "LineTotal" not in df.columns:
    df["LineTotal"] = df["Quantity"] * df["Price"]

# 5. Per-invoice basket stats
basket_per_invoice = (
    df.groupby(["InvoiceNo","CustomerID"], observed=True)["StockCode"]
      .count().reset_index(name="items_per_invoice")
)

# 6. Customer-level features
g = df.groupby("CustomerID", observed=True)
cust = (
    g.agg(
        tx_count     = ("InvoiceNo","nunique"),
        spend_sum    = ("LineTotal","sum"),
        item_qty_sum = ("Quantity","sum"),
        last_date    = ("InvoiceDate","max"),
    )
    .reset_index()
    .rename(columns={"CustomerID":"customerid"})
)

# Recency in days
max_date = pd.to_datetime(df["InvoiceDate"].max())
cust["recency_days"] = (max_date - pd.to_datetime(cust["last_date"])).dt.total_seconds() / 86400.0
cust = cust.drop(columns=["last_date"])

# Mean basket size per customer
basket_mean = (
    basket_per_invoice.groupby("CustomerID", observed=True)["items_per_invoice"]
      .mean().reset_index().rename(columns={"CustomerID":"customerid","items_per_invoice":"basket_size_mean"})
)

# Assemble feature table
feat = cust.merge(basket_mean, on="customerid", how="left")
feat["basket_size_mean"] = feat["basket_size_mean"].fillna(0.0)

# RFM-style score (invert recency because lower is better)
r_rank = feat["recency_days"].rank(ascending=False, pct=True)
f_rank = feat["tx_count"].rank(ascending=True,  pct=True)
m_rank = feat["spend_sum"].rank(ascending=True,  pct=True)
feat["RFM_Score"] = (1 - r_rank)*0.34 + f_rank*0.33 + m_rank*0.33

# Tidy numerics
num_cols = ["tx_count","spend_sum","item_qty_sum","basket_size_mean","recency_days","RFM_Score"]
feat[num_cols] = feat[num_cols].apply(pd.to_numeric, errors="coerce")
feat = feat.dropna(subset=["tx_count","spend_sum","recency_days"]).reset_index(drop=True)

# Quick summary
print(f"Rows after cleaning: {len(df):,}")
print(f"Unique customers: {df['CustomerID'].nunique():,}")
print(f"Date range: {df['InvoiceDate'].min()} → {df['InvoiceDate'].max()}")
print(f"Features table: {len(feat):,} customers, columns = {list(feat.columns)}")

# Aliases used later in the notebook
features = feat.copy()


Rows after cleaning: 805,549
Unique customers: 5,878
Date range: 2009-12-01 07:45:00 → 2011-12-09 12:50:00
Features table: 5,878 customers, columns = ['customerid', 'tx_count', 'spend_sum', 'item_qty_sum', 'recency_days', 'basket_size_mean', 'RFM_Score']


In [7]:
# Context figures for slides/poster (uses df and features from previous cells)
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

feat = features  # alias for this plotting cell

FIGS = Path(CFG["paths"]["figures_dir"]) if "CFG" in globals() else Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def savefig(name, dpi=160, tight=True):
    if tight: plt.tight_layout()
    plt.savefig(FIGS / name, dpi=dpi, bbox_inches="tight")
    plt.close()

# Daily transactions (seasonality)
daily = df.set_index("InvoiceDate").resample("D").size()
plt.figure(figsize=(8,3))
daily.plot()
plt.title("Daily transaction counts")
plt.xlabel("Date"); plt.ylabel("# Transactions")
savefig("daily_tx.png")

# Top 15 SKUs by line count
(df["StockCode"].astype(str).value_counts().head(15)).plot(kind="bar", figsize=(8,3))
plt.title("Top 15 SKUs by line count")
plt.xlabel("SKU"); plt.ylabel("Lines")
savefig("top_skus.png")

# Heavy-tail histograms (clipped at 99th percentile)
core = feat.copy()
clip = {
    "spend_sum":        core["spend_sum"].quantile(.99),
    "item_qty_sum":     core["item_qty_sum"].quantile(.99),
    "basket_size_mean": core["basket_size_mean"].quantile(.99),
}
fig, ax = plt.subplots(1,3, figsize=(12,3.5))
for i,(col,q) in enumerate(clip.items()):
    ax[i].hist(core[col].clip(upper=q), bins=40)
    ax[i].set_title(col)
fig.suptitle("Core distributions (clipped for readability)")
savefig("hists_core.png")

# Price × Quantity hexbin (log scale for counts)
from matplotlib.colors import LogNorm
p = df["Price"].clip(lower=0, upper=df["Price"].quantile(0.99))
q = df["Quantity"].clip(lower=0, upper=df["Quantity"].quantile(0.99))

fig, ax = plt.subplots(figsize=(7.2, 6))
hb = ax.hexbin(p, q, gridsize=60, mincnt=1, norm=LogNorm())
ax.set_xlabel("Unit Price"); ax.set_ylabel("Quantity")
ax.set_title("Price vs Quantity (clipped, log scale)")
cb = fig.colorbar(hb, ax=ax); cb.set_label("Count (log scale)")
savefig("price_qty_hex.png")
plt.close(fig)

# Pareto: cumulative customers vs cumulative spend
s = feat.sort_values("spend_sum", ascending=False)["spend_sum"].to_numpy()
cum = s.cumsum()/s.sum()
x = (np.arange(len(s))+1)/len(s)
plt.figure(figsize=(5,4))
plt.plot(x, cum)
plt.axvline(0.2, color="k", ls="--")
plt.axhline(cum[int(0.2*len(s))], color="k", ls="--")
plt.title("Pareto: cumulative customers vs cumulative spend")
plt.xlabel("Customer share"); plt.ylabel("Spend share")
savefig("pareto_spend.png")


In [ ]:
# Baseline Clustering — KMeans Sweep on Features (Comparative Metrics)
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

num_cols = [c for c in features.columns if features[c].dtype.kind in "if" and c.lower()!="customerid"]
X_feat = features[num_cols].fillna(0).to_numpy()
Xs_feat = StandardScaler().fit_transform(X_feat)

def sweep_metrics(Xs, title="features"):
    ks, sil, db, ch = [], [], [], []
    for k in range(CFG["clustering"]["kmeans"]["kmin"], CFG["clustering"]["kmeans"]["kmax"]+1):
        km = KMeans(n_clusters=k, n_init=CFG["clustering"]["kmeans"]["n_init"], random_state=CFG["random_seed"]).fit(Xs)
        y = km.labels_
        ks.append(k)
        sil.append(silhouette_score(Xs, y))
        db.append(davies_bouldin_score(Xs, y))
        ch.append(calinski_harabasz_score(Xs, y))
    plt.figure(figsize=(7,3))
    plt.plot(ks, sil, "-o", label="Silhouette (↑)")
    plt.plot(ks, db,  "-o", label="Davies–Bouldin (↓)")
    plt.plot(ks, ch,  "-o", label="Calinski–Harabasz (↑)")
    plt.title(f"Clustering metrics vs k — {title}")
    plt.xlabel("k"); plt.legend()
    savefig(f"silhouette_k_{title}.png")
    best_k = ks[int(np.argmax(sil))]
    return best_k, pd.DataFrame({"k":ks, "silhouette":sil, "davies_bouldin":db, "calinski_harabasz":ch})

best_k_feat, met_feat = sweep_metrics(Xs_feat, "features")

# Fit final baseline and export labels
km_feat = KMeans(n_clusters=best_k_feat, n_init=CFG["clustering"]["kmeans"]["n_init"],
                 random_state=CFG["random_seed"]).fit(Xs_feat)
labels_feat = pd.DataFrame({"customerid": features["customerid"], "label": km_feat.labels_})
labels_feat.to_csv(PROC / "kmeans_labels.csv", index=False)

# Profile heatmap (legible differences)
def profile_heatmap(df_features, labels_df, outname, title):
    df = df_features.merge(labels_df, on="customerid", how="left")
    numc = [c for c in df.columns
            if df[c].dtype.kind in "if" and c not in ("customerid", "label")]
    prof = df.groupby("label")[numc].mean()
    profz = (prof - prof.mean())/prof.std(ddof=0)
    plt.figure(figsize=(6,2.8))
    plt.imshow(profz, aspect="auto", cmap="coolwarm")
    plt.yticks(range(len(profz.index)), [f"C{int(c)}" for c in profz.index])
    plt.xticks(range(len(numc)), numc, rotation=45, ha="right")
    plt.title(title); plt.colorbar(label="z-score")
    savefig(outname)

profile_heatmap(features, labels_feat, "cluster_profile_heatmap_features.png", "Profiles — features")

# Auto-select the variant to PRESENT on the poster
scoreboard = []
if 'met_feat' in globals():
    scoreboard.append(("features", float(met_feat.loc[met_feat['k']==best_k_feat, 'silhouette'])))
if 'met_ae' in globals():
    scoreboard.append(("ae", float(met_ae.loc[met_ae['k']==best_k_ae, 'silhouette'])))
if 'met_seq' in globals():
    scoreboard.append(("seq", float(met_seq.loc[met_seq['k']==best_k_seq, 'silhouette'])))

scoreboard = sorted(scoreboard, key=lambda t: -t[1])
assert len(scoreboard) > 0, "No clustering metrics available yet."
PRES_VARIANT = scoreboard[0][0]
print("Presenting variant:", PRES_VARIANT, "silhouette:", scoreboard[0][1])

# Dynamic label-frame map (only include what exists)
variants = {}
if 'labels_feat' in globals(): variants['features'] = labels_feat
if 'labels_ae'   in globals(): variants['ae'] = labels_ae
if 'labels_seq'  in globals(): variants['seq'] = labels_seq
labels_df = variants[PRES_VARIANT]   # <-- use this for both plots below

# Segment lift: spend per customer (ranked, annotated)
Z = features.merge(labels_df, on="customerid", how="inner")

overall = Z["spend_sum"].mean()
lift = (Z.groupby("label", as_index=False)["spend_sum"].mean()
          .rename(columns={"spend_sum":"mean_spend"}))
lift["lift"] = lift["mean_spend"] / overall
lift = lift.sort_values("lift", ascending=False, ignore_index=True)
lift["segment"] = [f"S{i+1}" for i in range(len(lift))]

fig, ax = plt.subplots(figsize=(7.2, 4.2))
ax.bar(lift["segment"], lift["lift"])
ax.axhline(1.0, color="k", ls="--", lw=1)
ax.set_ylabel("Spend per customer (× overall)")
ax.set_title("Segment lift: spend per customer")
for i, (_, r) in enumerate(lift.iterrows()):
    ax.text(i, r["lift"] + 0.03, f"{r['lift']:.2f}×", ha="center", va="bottom", fontsize=11)
ax.set_ylim(0, max(1.1, lift["lift"].max() * 1.15))
ax.set_xlabel("Segment (ranked by lift)")
savefig("segment_lift_spend.png")
plt.close(fig)

# Re-engagement risk: share of “quiet” customers (≥ 75th pct recency)
Z = features.merge(labels_df, on="customerid", how="inner")
cut = np.nanpercentile(Z["recency_days"], 75)
Z["quiet"] = (Z["recency_days"] >= cut).astype(int)

quiet_share = (Z.groupby("label", as_index=False)["quiet"].mean()
                 .rename(columns={"quiet":"share_quiet"}))
quiet_share = quiet_share.sort_values("share_quiet", ascending=False, ignore_index=True)
quiet_share["segment"] = [f"S{i+1}" for i in range(len(quiet_share))]
baseline = Z["quiet"].mean()

fig, ax = plt.subplots(figsize=(7.2, 4.2))
ax.barh(quiet_share["segment"], quiet_share["share_quiet"])
ax.axvline(baseline, color="k", ls="--", lw=1)
ax.set_xlabel("Share quiet (≥ 75th %ile recency)")
ax.set_title("Re-engagement risk by segment")
for y, p in enumerate(quiet_share["share_quiet"]):
    ax.text(p + 0.01, y, f"{p*100:.1f}%", va="center", fontsize=11)
ax.set_xlim(0, max(quiet_share["share_quiet"].max()*1.15, baseline*1.15))
ax.invert_yaxis()  # S1 at top
savefig("segment_reengage_share.png")
plt.close(fig)

# Export metrics
met_feat.to_csv(PROC / "metrics_features.csv", index=False)
print("Baseline K-means (features) → best k =", best_k_feat,
      "sil=", float(met_feat.loc[met_feat["k"]==best_k_feat, "silhouette"]))


Presenting variant: features silhouette: 0.8824713663751557
Baseline K-means (features) → best k = 2 sil= 0.8824713663751557


In [9]:
# Tabular Autoencoder (Embedding + Comparative Clustering)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.cluster import KMeans

# Paths (in case this cell runs standalone)
from pathlib import Path
PROC = Path(CFG["paths"]["processed_dir"]) if "CFG" in globals() else Path("data/processed")
PROC.mkdir(parents=True, exist_ok=True)

def build_tab_ae(input_dim, bottleneck=8, l2=1e-4, p_drop=0.10):
    reg = keras.regularizers.l2(l2)
    inp = keras.Input(shape=(input_dim,))
    x = layers.BatchNormalization()(inp)
    x = layers.Dense(64, activation="relu", kernel_regularizer=reg)(x)
    x = layers.Dropout(p_drop)(x)
    x = layers.Dense(32, activation="relu", kernel_regularizer=reg)(x)
    z = layers.Dense(bottleneck, activation="linear", name="z")(x)
    x = layers.Dense(32, activation="relu", kernel_regularizer=reg)(z)
    x = layers.Dropout(p_drop)(x)
    x = layers.Dense(64, activation="relu", kernel_regularizer=reg)(x)
    out = layers.Dense(input_dim, activation="linear")(x)
    ae = keras.Model(inp, out)
    enc = keras.Model(inp, z)
    ae.compile(optimizer=keras.optimizers.Adam(1e-3), loss="mse")
    return ae, enc

# Config shims / defaults
ae_cfg   = (CFG.get("ae", {}) if "CFG" in globals() else {})
bottleneck = ae_cfg.get("bottleneck", 8)
l2         = ae_cfg.get("l2", 1e-4)
p_drop     = ae_cfg.get("dropout", 0.10)      # map 'dropout' -> p_drop
patience   = ae_cfg.get("patience", 8)
epochs     = ae_cfg.get("epochs", 50)
batch      = ae_cfg.get("batch", 256)

# Xs_feat must exist from your scaling cell
Xs = Xs_feat

ae, enc = build_tab_ae(Xs.shape[1], bottleneck=bottleneck, l2=l2, p_drop=p_drop)
cb = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=patience, restore_best_weights=True)]
hist = ae.fit(Xs, Xs, validation_split=0.2, epochs=epochs, batch_size=batch, verbose=0, callbacks=cb)

# Loss curve
plt.figure(figsize=(6,3))
plt.plot(hist.history["loss"], label="train")
plt.plot(hist.history["val_loss"], label="val")
plt.legend(); plt.title("AE training (MSE)")
savefig("ae_loss.png")

# Embeddings -> cluster -> metrics
E = enc.predict(Xs, verbose=0)
best_k_ae, met_ae = sweep_metrics(E, "ae")    # Assumes your sweep_metrics() is defined earlier

km_ae = KMeans(n_clusters=best_k_ae, n_init=CFG["clustering"]["kmeans"]["n_init"], random_state=CFG["random_seed"]).fit(E)
labels_ae = pd.DataFrame({"customerid": features["customerid"], "label": km_ae.labels_})
labels_ae.to_csv(PROC / "embed_kmeans_labels.csv", index=False)

# Profile heatmap for slides (assumes profile_heatmap() defined earlier)
profile_heatmap(features, labels_ae, "cluster_profile_heatmap_ae.png", "Profiles — AE")

met_ae.to_csv(PROC / "metrics_ae.csv", index=False)
print("AE clustering → best k =", best_k_ae, "  sil=",
      met_ae.loc[met_ae["k"]==best_k_ae, "silhouette"].values[0])





AE clustering → best k = 2   sil= 0.90397596


In [10]:
# Sequence Encoder (LSTM Next-Token) + Comparative Clustering (habit vs variety)
import numpy as np
import pandas as pd
from collections import Counter
from pathlib import Path
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.cluster import KMeans

# Helpers / Shims 
def _rename_first_match(df, std, candidates):
    for c in candidates:
        if c in df.columns:
            return df if c == std else df.rename(columns={c: std})
    raise KeyError(f"Required column '{std}' not found. Looked for: {candidates}. "
                   f"Available (head): {list(df.columns)[:15]}")

# Use cleaned dataframe
if "df" not in globals():
    raise NameError("Missing `df`. Run the loading/cleaning + feature engineering cells first.")

# Normalize the few names we need here (safe no-ops if already standard)
df = df.copy()
df = _rename_first_match(df, "CustomerID",  ["CustomerID", "Customer ID", "customerid"])
df = _rename_first_match(df, "InvoiceDate", ["InvoiceDate", "Invoice Date", "date"])
df = _rename_first_match(df, "StockCode",   ["StockCode", "Stock Code", "SKU", "ProductCode", "Product Code", "Description"])

# Strong types we rely on
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
df = df.dropna(subset=["CustomerID", "InvoiceDate"]).copy()
df["CustomerID"] = pd.to_numeric(df["CustomerID"], errors="coerce").round().astype("Int64")
df = df.dropna(subset=["CustomerID"]).astype({"CustomerID":"int64"})
df["StockCode"] = df["StockCode"].astype(str).str.strip()

# Build per-customer SKU sequences in time order
cust_seq = (df.sort_values(["CustomerID","InvoiceDate"])
              .groupby("CustomerID", observed=True)["StockCode"]
              .apply(list))

# Vocabulary: keep frequent tokens
seq_cfg   = CFG.get("seq", {}) if "CFG" in globals() else {}
min_count = seq_cfg.get("min_count", 5)
max_vocab = seq_cfg.get("max_vocab", 2000)
seq_len   = seq_cfg.get("seq_len", 50)
batch     = seq_cfg.get("batch", 256)
epochs    = seq_cfg.get("epochs", 10)

cnt  = Counter([t for seq in cust_seq for t in seq])
keep = [t for t,c in cnt.items() if c >= min_count]
keep = [t for t,_ in sorted(((t,cnt[t]) for t in keep), key=lambda z: -z[1])][:max_vocab]
tok2id = {t:i+2 for i,t in enumerate(keep)}   # 0=PAD, 1=UNK
PAD, UNK = 0, 1

def encode(seq): return [tok2id.get(t, UNK) for t in seq]

# Sliding windows for next-token task
X_tok, y_tok = [], []
for seq in cust_seq:
    ids = encode(seq)
    if len(ids) < 2: 
        continue
    for i in range(1, len(ids)):
        window = ids[max(0, i-seq_len):i]
        if len(window) < 2:
            continue
        X_tok.append(window[-seq_len:])
        y_tok.append(ids[i])

if len(X_tok) == 0:
    raise RuntimeError("No sequences long enough to build training samples. "
                       "Try lowering seq_len or min_count in CFG['seq'].")

X_tok = pad_sequences(X_tok, maxlen=seq_len, padding="pre", truncating="pre", value=PAD)
y_tok  = np.array(y_tok, dtype="int32")

# Optional cap to keep runtime/memory in check
max_samples = seq_cfg.get("max_samples")
if max_samples and len(X_tok) > max_samples:
    rng = np.random.default_rng(1337)
    take = rng.choice(len(X_tok), size=max_samples, replace=False)
    X_tok, y_tok = X_tok[take], y_tok[take]

# Train/val split
n   = len(X_tok)
idx = np.arange(n); np.random.shuffle(idx)
cut = int(0.8*n)
tr, va = idx[:cut], idx[cut:]
Xtr, Xva, ytr, yva = X_tok[tr], X_tok[va], y_tok[tr], y_tok[va]

# LSTM next-token classifier (penultimate layer as embedding)
vocab_size = len(tok2id) + 2
embed_dim  = 64
lstm_dim   = 64

inp = keras.Input(shape=(seq_len,), dtype="int32")
x   = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(inp)
x   = layers.SpatialDropout1D(0.1)(x)
x   = layers.LSTM(lstm_dim, return_sequences=False)(x)
z   = layers.Dropout(0.2, name="seq_embedding")(x)
out = layers.Dense(vocab_size, activation="softmax")(z)

seq_model = keras.Model(inp, out)
seq_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
cb2 = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True)]

hist_seq = seq_model.fit(Xtr, ytr, validation_data=(Xva, yva),
                         epochs=epochs, batch_size=batch, verbose=0, callbacks=cb2)

# Training curves (for slides/poster)
plt.figure(figsize=(6,3))
plt.plot(hist_seq.history["loss"], label="train")
plt.plot(hist_seq.history["val_loss"], label="val")
plt.legend(); plt.title("LSTM training (next-token)")
savefig("lstm_loss.png")

# Per-customer embeddings: embed last seq_len tokens per customer
enc_seq = keras.Model(seq_model.input, z)

cust_ids, cust_emb = [], []
for cid, seq in cust_seq.items():
    ids = encode(seq)
    if not ids:
        continue
    x = pad_sequences([ids[-seq_len:]], maxlen=seq_len, padding="pre", truncating="pre", value=PAD)
    e = enc_seq.predict(x, verbose=0)[0]
    cust_ids.append(cid); cust_emb.append(e)

seq_emb = pd.DataFrame(cust_emb, columns=[f"e{i}" for i in range(len(cust_emb[0]))])
seq_emb.insert(0, "customerid", cust_ids)

# Cluster sequence embeddings and record metrics for comparison
E2 = seq_emb.drop(columns=["customerid"]).to_numpy()
best_k_seq, met_seq = sweep_metrics(E2, "seq")  # Uses earlier sweep_metrics

km_seq = KMeans(
    n_clusters=best_k_seq, 
    n_init=CFG["clustering"]["kmeans"]["n_init"], 
    random_state=CFG["random_seed"]
).fit(E2)

labels_seq = pd.DataFrame({"customerid": seq_emb["customerid"], "label": km_seq.labels_})
labels_seq.to_csv(Path(CFG["paths"]["processed_dir"]) / "embed_kmeans_labels_seq.csv", index=False)

profile_heatmap(features, labels_seq, "cluster_profile_heatmap_seq.png", "Profiles — Sequence")

met_seq.to_csv(Path(CFG["paths"]["processed_dir"]) / "metrics_seq.csv", index=False)
print("SEQ clustering → best k =", best_k_seq,
      "sil=", met_seq.loc[met_seq["k"]==best_k_seq, "silhouette"].values[0])




SEQ clustering → best k = 2 sil= 0.08792641


In [ ]:
# Stability vs seed (boxplots of silhouette scores across variants)
import numpy as np, matplotlib.pyplot as plt

# assume you already computed arrays sil_feat, sil_ae, sil_seq
labels = ["features", "ae", "seq"]
data   = [sil_feat, sil_ae, sil_seq]

fig, ax = plt.subplots(figsize=(5.6, 3.4))
bp = ax.boxplot(data, patch_artist=True, labels=labels, widths=0.55,
                boxprops=dict(facecolor="#3B82F6", alpha=0.35, linewidth=1.2),
                medianprops=dict(color="#1F2937", linewidth=1.5),
                whiskerprops=dict(color="#4B5563"),
                capprops=dict(color="#4B5563"),
                flierprops=dict(marker=".", markersize=3, markerfacecolor="#3B82F6", alpha=0.5))
ax.set_ylabel("Silhouette")
ax.set_title("Stability vs seed")
plt.tight_layout()
plt.savefig(FIGS / "stability_box.png", dpi=300, bbox_inches="tight")
plt.close(fig)


NameError: name 'Xs_feat' is not defined

In [1]:
# Anomaly Detection figs (AE log-MSE + IF×AE hexbin)
import numpy as np, pandas as pd, matplotlib.pyplot as plt, matplotlib as mpl
from pathlib import Path
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogFormatter
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import ConnectionPatch

try:
    FIGS
except NameError:
    FIGS = Path("reports/figures"); FIGS.mkdir(parents=True, exist_ok=True)

try:
    savefig
except NameError:
    def savefig(name, dpi=300, tight=True):
        if tight: plt.tight_layout()
        plt.savefig(FIGS / name, dpi=dpi, bbox_inches="tight"); plt.close()

# If 'scores' isn't in memory, try to load it.
if "scores" not in globals():
    cand = Path("data/processed/anomaly_scores.csv")
    if not cand.exists():
        raise RuntimeError("Need a DataFrame 'scores' with columns ['if_score','ae_mse'].")
    scores = pd.read_csv(cand)


# 1. Agreement plot (IF vs AE) — hexbin with % and labeled thresholds
df_agree = scores.copy()
df_agree["ae_log"] = np.log10(np.clip(df_agree["ae_mse"].values, np.finfo(float).eps, None))

q = 0.97  # top 3% thresholds
th_if  = np.quantile(df_agree["if_score"], q)
th_aeL = np.quantile(df_agree["ae_log"],  q)

fig, ax = plt.subplots(figsize=(7.4, 5.6))
hb = ax.hexbin(
    df_agree["if_score"], df_agree["ae_log"],
    gridsize=45, mincnt=1, cmap="viridis", norm=LogNorm()
)
cb = fig.colorbar(hb, ax=ax, format=LogFormatter(10))
cb.set_label("Count (log scale)")

# Threshold lines + tiny labels
ax.axvline(th_if,  color="k", ls="--", lw=1)
ax.axhline(th_aeL, color="k", ls="--", lw=1)
ax.text(th_if, ax.get_ylim()[1], " IF q97 ", ha="left", va="top", fontsize=10, rotation=90,
        bbox=dict(fc="white", ec="0.6", pad=0.2))
ax.text(ax.get_xlim()[1], th_aeL, " AE q97 ", ha="right", va="bottom", fontsize=10,
        bbox=dict(fc="white", ec="0.6", pad=0.2))

# Agreement count + percent in top-right quadrant
mask_agree = (df_agree["if_score"] >= th_if) & (df_agree["ae_log"] >= th_aeL)
n_agree = int(mask_agree.sum())
pct_agree = 100.0 * n_agree / len(df_agree)
ax.text(
    0.02, 0.98,
    f"Top-right (agree) = {n_agree}  ({pct_agree:.1f}%)",
    transform=ax.transAxes, va="top", ha="left", fontsize=12,
    bbox=dict(boxstyle="round,pad=0.25", fc="white", ec="0.6")
)

ax.set_xlabel("Isolation Forest score")
ax.set_ylabel("log10(AE reconstruction MSE)")
ax.set_title("Anomaly agreement: IF vs AE")
savefig("anomaly_agreement_hex.png")

# 2. AE reconstruction error — log histogram with lower-right inset
xlog = np.log10(np.clip(scores["ae_mse"].values, np.finfo(float).eps, None))
p95, p99 = np.percentile(xlog, [95, 99])

fig, ax = plt.subplots(figsize=(10, 5.2))
n, bins, _ = ax.hist(xlog, bins=60, color="#2B6CB0", alpha=0.92,
                     edgecolor="white", linewidth=0.3)

# 95th / 99th markers + light tail shading
ax.axvline(p95, color="k", ls="--", lw=1)
ax.axvline(p99, color="k", ls=":",  lw=1)
ax.text(p95, ax.get_ylim()[1]*0.94, "95th %", ha="right", va="top", fontsize=11)
ax.text(p99, ax.get_ylim()[1]*0.88, "99th %", ha="right", va="top", fontsize=11)
ax.fill_between(
    bins, 0, np.interp(bins, (bins[:-1]+bins[1:])/2, n),
    where=bins >= p99, color="#E53E3E", alpha=0.22, step="pre"
)

ax.set_xlabel("log10(AE reconstruction MSE)")
ax.set_ylabel("Count")
ax.set_title("AE reconstruction error (log scale)")

# Inset: zoom the tail (lower-right, unobtrusive)
axins = inset_axes(ax, width="34%", height="58%", loc="lower right",
                   bbox_to_anchor=(0.02, 0.05, 1, 1), bbox_transform=ax.transAxes, borderpad=0.6)
axins.hist(xlog, bins=60, color="#2B6CB0", alpha=0.92, edgecolor="white", linewidth=0.3)
axins.set_xlim(p95, xlog.max())
axins.set_yticks([]); axins.set_xticks([])
axins.set_title("Tail zoom", fontsize=9)

# Optional: a subtle connector line from inset to main
con = ConnectionPatch(
    xyA=(p99, 0), coordsA=ax.transData,
    xyB=(axins.get_xlim()[0], 0), coordsB=axins.transData,
    color="0.5", lw=0.8, alpha=0.7
)
fig.add_artist(con)

savefig("ae_mse_hist.png")


C:\Users\mttng\AppData\Local\Temp\ipykernel_15600\2899549644.py:18: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  if tight: plt.tight_layout()


In [13]:
# Exports For What to drop into slides/poster
export_list = [
    "daily_tx.png","top_skus.png","hists_core.png","price_qty_hex.png","pareto_spend.png",
    "silhouette_k_features.png","cluster_profile_heatmap_features.png","segment_lift_spend.png","segment_reengage_share.png",
    "ae_loss.png","silhouette_k_ae.png","cluster_profile_heatmap_ae.png",
    "lstm_loss.png","silhouette_k_seq.png","cluster_profile_heatmap_seq.png",
    "stability_box.png",
    "if_score_hist.png","ae_mse_hist.png","anomaly_agreement_scatter.png","anomaly_cume_spend.png",
]
missing = [f for f in export_list if not (FIGS/f).exists()]
print("Figures ready:", len(export_list)-len(missing), " / ", len(export_list))
if missing:
    print("Missing:", missing)

print("\nCSV/Parquet in:", PROC)
print(sorted([p.name for p in PROC.glob("*.csv")]))


Figures ready: 20  /  20

CSV/Parquet in: C:\Users\mttng\Downloads\retail-segmentation-anomalies\data\processed
['anomaly_scores.csv', 'anomaly_top10.csv', 'embed_kmeans_labels.csv', 'embed_kmeans_labels_seq.csv', 'kmeans_labels.csv', 'metrics_ae.csv', 'metrics_features.csv', 'metrics_seq.csv']
